<a href="https://colab.research.google.com/github/Osgil/Data_analyst/blob/main/Analysis_of_variance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
import pandas as pd
import scipy.stats as st
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [12]:
df = pd.read_csv('/content/drive/MyDrive/5_task_1.csv') # Считываем данные в 
                                                        # датафрейм  

В столбце group показан тип картинок, которые использовались для показа пользо-
вателям, в столбце events - количество покупок товаров с тем или иным разрешением картинки. Задача - определить наиболее удачный формат картинок.

In [20]:
df.head()

,id,group,events
0,16046,A,14
1,18712,A,41
2,3268,A,30
3,22633,A,37
4,28071,A,38


In [28]:
df.shape

(30000, 3)

In [78]:
df.isna().sum() # пропусков не обнаружено

id        0
group     0
events    0
dtype: int64

Проверим на гомогенность дисперсию групп c помощью теста Левена.

In [17]:
df.group.unique() # Всего имеется три группы.

array(['A', 'B', 'C'], dtype=object)

In [93]:
# Разобьем датафрейм на группы
a = df.query("group == 'A'").events
b = df.query("group == 'B'").events
c = df.query("group == 'C'").events

In [27]:
st.levene(a, b, c) # Нулевая гипотеза, что дисперсии гомогенны, не отклоняется 
                   # при 95% уровне значимости, p > 0.05

LeveneResult(statistic=2.2622596339318037, pvalue=0.1041327199523974)

Проверим, нормально ли распределены данные, с помощью теста Шапиро-Уилка

In [58]:
sub = df.sample(n=1000, random_state=1) # метод может работать не правильно на
                                         # больших данных, возьмем выборку
                                         # random_state значение 1 для воспро -
                                         # изводимости результатов.

In [59]:
st.shapiro(sub.query("group == 'A'").events) # данные в группе А распределены нормально

(0.9944230914115906, 0.24558229744434357)

In [56]:
st.shapiro(sub.query("group == 'B'").events) # данные в группе B распределены нормально

(0.9966618418693542, 0.7229400277137756)

In [61]:
st.shapiro(sub.query("group == 'C'").events) # данные в группе C распределены нормально

(0.9943881034851074, 0.27812638878822327)

In [62]:
# проведем тесты с полным набором данных

In [66]:
st.shapiro(df.query("group == 'A'").events) # в случае использования полного на-
                                            # мы бы отклонили нулевую гипотезу
                                            # теста Шапиро о нормальном распре-
                                            # делении данных для группы A 

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


(0.9989418387413025, 4.318870651331963e-06)

In [68]:
st.shapiro(df.query("group == 'B'").events) # и B

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


(0.9990161061286926, 1.0504894817131571e-05)

In [70]:
st.shapiro(df.query("group == 'C'").events) # и С тоже 

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


(0.9989210963249207, 3.3885160064528463e-06)

In [67]:
# Посмотрим, какие результаты покажет normaltest из библиотеки scipy

In [73]:
st.normaltest(df.query("group == 'A'").events)

NormaltestResult(statistic=2.1034599402238734, pvalue=0.3493328906085719)

In [74]:
st.normaltest(df.query("group == 'B'").events)

NormaltestResult(statistic=0.8410715800223014, pvalue=0.6566948749848232)

In [76]:
st.normaltest(df.query("group == 'C'").events) # normaltest определяет наши дан-
                                               # ные как нормальные, даже без 
                                               # использования выборок

NormaltestResult(statistic=3.243725809054571, pvalue=0.19753037588314626)

Проведем однофакторный тест Фишера

In [95]:
st.f_oneway(a, b, c) # отклоняем нулевую гипотезу о равентстве средних 

F_onewayResult(statistic=2886.166657438961, pvalue=0.0)

С помощью критерия Тьюки определим между какими группами есть статистически значимые различия

In [102]:
pairwise_tukeyhsd(df.events, df.group).summary() # различия между всеми 3 нашими
                                                 # группами стат.значимые
                                                 # как мы видим из столбца meandiff
                                                 # наиболее удачное разрешение кар-
                                                 # тинки - B.

group1,group2,meandiff,p-adj,lower,upper,reject
A,B,8.034,0.001,7.7038,8.3642,True
A,C,-2.1093,0.001,-2.4395,-1.7791,True
B,C,-10.1433,0.001,-10.4735,-9.8131,True
